In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from dask.distributed import Client
from seapopym.configuration.no_transport.parameter import ForcingParameters

# from seapopym.configuration.parameters.parameter_environment import (
#     ChunkParameter,
#     ClientParameter,
#     EnvironmentParameter,
# )
from seapopym.configuration.parameters.parameter_forcing import ForcingUnit
from seapopym.standard.units import StandardUnitsLabels

from seapopym_optimization import Observation, constraint
from seapopym_optimization.cost_function import NoTransportCostFunction
from seapopym_optimization.functional_groups import FunctionalGroupOptimizeNoTransport, Parameter
from seapopym_optimization.genetic_algorithm import GeneticAlgorithm, GeneticAlgorithmParameters
from seapopym_optimization.taylor_diagram import ModTaylorDiagram, generate_mod_taylor_diagram

xr.set_options(
    display_expand_attrs=False,
    display_expand_data_vars=False,
    display_expand_coords=False,
    display_expand_data=False,
)

In [2]:
path_to_forcing = "../../../1_data_processing/1_1_Forcing/data/1_products/all_stations_cmems_climato.zarr"
path_to_observed_npp_hot = "../../../1_data_processing/1_1_Forcing/data/1_products/Hot_observed_npp_climato.zarr"
path_to_obs = {
    "BATS": "../../../1_data_processing/1_1_Forcing/data/1_products/Bats_obs_zoo_climato_monthly_2002_2015.zarr",
    "HOT": "../../../1_data_processing/1_1_Forcing/data/1_products/Hot_obs_zoo_climato_monthly_2002_2015.zarr",
    "PAPA": "../../../1_data_processing/1_1_Forcing/data/1_products/Papa_obs_zoo_climato_monthly_2002_2015.zarr",
    "CALCOFI": "../../../1_data_processing/1_1_Forcing/data/1_products/Calcofi_obs_zoo_climato_monthly_2002_2015.zarr",
}
export_file_name = "Climato_all_4_stations_2_groups"

In [3]:
TIME_START = "2005-01-01"
TIME_END = "2007-01-01"
STABILIZATION_TIME = 12
SAVE = False

In [4]:
# client = Client(n_workers=2, threads_per_worker=1, memory_limit="16GB")
client = Client(n_workers=2, threads_per_worker=2, memory_limit="16GB")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 4,Total memory: 29.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61958,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 29.80 GiB
Comm: tcp://127.0.0.1:61965,Total threads: 2
Dashboard: http://127.0.0.1:61968/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:61961,


## Loading


### Forcing


In [5]:
forcing = xr.open_zarr(path_to_forcing)
forcing = forcing.sel(time=slice(TIME_START, TIME_END))
forcing["T"].attrs["units"] = StandardUnitsLabels.temperature.units
forcing.load()

<xarray.Dataset> Size: 774MB
Dimensions:              (time: 731, latitude: 60, longitude: 105, depth: 3)
Coordinates: (4)
Data variables: (13)
Attributes: (6)

Load observed npp in HOT.


In [6]:
observed_npp = xr.open_zarr(path_to_observed_npp_hot)
observed_npp = observed_npp.sel(time=slice(TIME_START, TIME_END))
observed_npp.load()

<xarray.Dataset> Size: 592B
Dimensions:          (time: 24, latitude: 1, longitude: 1)
Coordinates: (3)
Data variables: (2)
Attributes: (2)

Convert weekly npp observations to forcing calendar and use interpolation to fill in the gaps.


In [7]:
vgpm_hot = forcing.npp.sel(latitude=observed_npp.latitude, longitude=observed_npp.longitude)  # .data
observed_npp = observed_npp.interp_calendar(vgpm_hot.time).interpolate_na(
    dim="time", method="linear", fill_value="extrapolate"
)
observed_npp

<xarray.Dataset> Size: 18kB
Dimensions:          (latitude: 1, longitude: 1, time: 731)
Coordinates: (3)
Data variables: (2)
Attributes: (2)

Replace hot primary production in forcing.


In [ ]:
forcing.npp.sel(latitude=observed_npp.latitude, longitude=observed_npp.longitude).data = observed_npp["l12"].data

### Epipelagic layer


In [ ]:
epi_layer_depth = forcing["pelagic_layer_depth"].sel(depth=0)
epi_layer_depth = epi_layer_depth.resample(time="1D").mean()
epi_layer_depth.attrs["units"] = "meter"
epi_layer_depth = epi_layer_depth.pint.quantify()
epi_layer_depth

<!-- ## Observed NPP -->


### Observations


In [ ]:
with xr.set_options(keep_attrs=True):
    observations = {}
    for name_obs, path_obs in path_to_obs.items():
        obs = xr.open_zarr(path_obs).load()
        # Remove the X first months to let the model reach the stationary state.
        obs = obs.sel(time=slice(TIME_START, TIME_END)).isel(time=slice(STABILIZATION_TIME, None))
        obs = obs.dropna("time", how="all")
        obs = obs.pint.quantify()
        obs = obs * epi_layer_depth
        observations[name_obs] = obs[["day", "night"]].drop_vars("depth").pint.dequantify()
observations

## Send forcing to each core


In [ ]:
# forcing["T"].chunk()
# forcing["npp"].chunk()
# observations = [
#     Observation(name=name, observation=obs.chunk(), observation_type="monthly") for name, obs in observations.items()
# ]

In [ ]:
forcing["T"] = forcing["T"].astype(np.float32)
forcing["npp"] = forcing["npp"].astype(np.float32)

# forcing = forcing.chunk()
# observations = {name: obs.chunk() for name, obs in observations.items()}

observations = [
    Observation(name=name, observation=obs.astype(np.float32).load(), observation_type="monthly")
    for name, obs in observations.items()
]

Create structure for SeapoPym simulation.


In [ ]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=forcing["T"], resolution=1 / 12, timestep=1),
    primary_production=ForcingUnit(forcing=forcing["npp"], resolution=1 / 12, timestep=1),
)

## Setup the parameters and the cost function


In [ ]:
functional_groups = [
    FunctionalGroupOptimizeNoTransport(
        name="D1N1",
        day_layer=0,
        night_layer=0,
        energy_coefficient=Parameter("D1N1_energy_coefficient", 0.001, 0.4),
        tr_rate=Parameter("D1N1_tr_rate", -0.3, -0.001),
        tr_max=Parameter("D1N1_tr_max", 0, 50),
        inv_lambda_rate=Parameter("D1N1_inv_lambda_rate", -0.3, -0.001),
        inv_lambda_max=Parameter("D1N1_inv_lambda_max", 100, 200),
    ),
    FunctionalGroupOptimizeNoTransport(
        name="D2N1",
        day_layer=1,
        night_layer=0,
        energy_coefficient=Parameter("D2N1_energy_coefficient", 0.001, 0.4),
        tr_rate=Parameter("D2N1_tr_rate", -0.3, -0.001),
        tr_max=Parameter("D2N1_tr_max", 0, 50),
        inv_lambda_rate=Parameter("D2N1_inv_lambda_rate", -0.3, -0.001),
        inv_lambda_max=Parameter("D2N1_inv_lambda_max", 100, 200),
    ),
]

In [ ]:
# from seapopym.configuration.parameters.parameter_environment import EnvironmentParameter, ChunkParameter

cost_function = NoTransportCostFunction(
    functional_groups=functional_groups,
    forcing_parameters=forcing_parameters,
    observations=observations,
    normalized_mse=True,
    root_mse=True,
    # environment_parameters=EnvironmentParameter(chunk=ChunkParameter(functional_group=2)),
)

Optional : Compute max memory size + execution time.


In [ ]:
# from time import time

# from seapopym_optimization.wrapper import FunctionalGroupGeneratorNoTransport, model_generator_no_transport

# model = model_generator_no_transport(
#     forcing_parameters=forcing_parameters,
#     fg_parameters=FunctionalGroupGeneratorNoTransport(
#         [
#             [0, 0, 0.1, 50, -0.1, 150, -0.1],
#             [0, 1, 0.1, 50, -0.1, 150, -0.1],
#         ],
#         groups_name=["D1N1", "D2N1"],
#     ),
# )


# time_start = time()
# model.run()
# time_end = time()

# print(model.expected_memory_usage)
# print(f"Time to run the model: {time_end - time_start} seconds")

Set the genetic algorithm meta parameters.


In [ ]:
genetic_algo_parameters = GeneticAlgorithmParameters(
    MUTPB=0.30,
    INDPB=0.2,
    ETA=5,
    CXPB=0.7,
    NGEN=1,
    POP_SIZE=4000,
    cost_function_weight=(-1, -1, -1, -1),
)

Add a constraint to limit the total of energy transfert coefficient to 100%.


In [ ]:
constraint_energy = constraint.ConstraintNoTransportEnergyCoefficient(
    parameters_name=["D1N1_energy_coefficient", "D2N1_energy_coefficient"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

Finaly, create the Genetic Algorithm.


In [ ]:
genetic_algo = GeneticAlgorithm(
    cost_function=cost_function,
    parameter_genetic_algorithm=genetic_algo_parameters,
    constraint=[constraint_energy],
    client=client,
    logbook_path=f"{export_file_name}_logbook.json",
)

And watch the magic on the Dask dashboard :


In [ ]:
genetic_algo.client

## Run the optimization


In [ ]:
viewer = genetic_algo.optimize()

## Optimization statistics


In [ ]:
viewer.hall_of_fame.head(10)

In [ ]:
viewer.fitness_evolution()

In [ ]:
viewer.parameters_standardized_deviation()

In [ ]:
viewer.parameters_scatter_matrix(nbest=10_000)

In [ ]:
fig = viewer.box_plot(5, nbest=1000)
fig.show()

In [ ]:
groups = [
    ["D1N1_energy_coefficient", "D1N1_tr_rate", "D1N1_tr_max", "D1N1_inv_lambda_rate", "D1N1_inv_lambda_max"],
    ["D2N1_energy_coefficient", "D2N1_tr_rate", "D2N1_tr_max", "D2N1_inv_lambda_rate", "D2N1_inv_lambda_max"],
]

fig = viewer.parallel_coordinates(nbest=2_000, unselected_opacity=0, parameter_groups=groups, uniformed=True)

for group in fig:
    display(group)

In [ ]:
if SAVE:
    for i, trace in enumerate(fig):
        trace.write_html(f"Parallel_coordinates_{export_file_name}_{i}.html")

# Plots


### Time series of X best individuals


In [ ]:
all_figures = viewer.time_series(10, title=list(path_to_obs.keys()), client=client)
for figure in all_figures:
    figure.update_layout(width=1000, height=600)
    figure.show()

In [ ]:
if SAVE:
    fig.write_html(f"Biomass_best_individuals_{export_file_name}.html")

### Correlation matrix of X best individuals


In [ ]:
fig = viewer.parameters_correlation_matrix(10_000)
fig.update_layout(width=1000, height=1000)

In [ ]:
if SAVE:
    fig.write_html(f"Correlation_best_individuals_{export_file_name}.html")

In [ ]:
import plotly.express as px
import numpy as np


def weighted_correlation_matrix(X, w):
    """
    Calcule la matrice de corrélation pondérée de Pearson pour un ensemble de variables.

    Paramètres :
    X : array-like (NxM) - Matrice des observations (N observations, M variables)
    w : array-like (N,)  - Poids associés à chaque observation

    Retourne :
    R_w : array (MxM) - Matrice de corrélation pondérée (M variables x M variables)
    """
    X = np.array(X)
    w = np.array(w).reshape(-1, 1)  # Convertir en colonne (N,1) si nécessaire

    # Nombre de variables
    M = X.shape[1]

    # Moyennes pondérées pour chaque variable
    mean_w = np.sum(w * X, axis=0) / np.sum(w)

    # Écarts-types pondérés pour chaque variable
    std_w = np.sqrt(np.sum(w * (X - mean_w) ** 2, axis=0))

    # Matrice de covariance pondérée
    cov_w = np.dot((X - mean_w).T, w * (X - mean_w))

    # Matrice de corrélation pondérée
    R_w = cov_w / np.outer(std_w, std_w)

    # Nan sur la diagonale
    np.fill_diagonal(R_w, np.nan)

    return R_w


X = viewer.hall_of_fame.to_numpy()[:, :-1]
# Solution polynomiale
# w = np.abs(1 / viewer.hall_of_fame["fitness"].to_numpy())
# Solution linéaire
w = viewer.hall_of_fame["fitness"].to_numpy()
w = (-w) + np.max(w)

R_w = weighted_correlation_matrix(X, w)

fig = px.imshow(
    R_w,
    width=800,
    height=700,
    text_auto=False,
    aspect="auto",
    color_continuous_scale=[[0, "blue"], [0.5, "white"], [1, "red"]],
    zmin=-1,
    zmax=1,
    x=viewer.hall_of_fame.columns[:-1],
    y=viewer.hall_of_fame.columns[:-1],
)
# enlève le fond et la grille
fig.update_xaxes(showgrid=False, showline=False, zeroline=False)
fig.show()

In [ ]:
plt.plot(w)
plt.plot(viewer.hall_of_fame["fitness"].to_numpy())
plt.legend(["poids", "fitness"])
plt.xlabel("Individu")
plt.ylabel("Valeur")
plt.title("Poids et fitness des individus : w = -fitness + max(fitness)")

In [ ]:
fig, ax = plt.subplots()
ax.plot(w)
ax2 = ax.twinx()

ax2.plot(viewer.hall_of_fame["fitness"].to_numpy(), color="red")
plt.legend(["poids", "fitness"])
plt.xlabel("Individu")
ax.set_ylabel("Poids")
ax2.set_ylabel("Fitness")
ax2.set_yscale("log")
plt.title("Poids et fitness des individus : w = absolute(1 / fitness)")

### Taylor Diagram


In [ ]:
fig = viewer.taylor_diagram(1, client=client)
# dont show legend
fig.update_layout(showlegend=False)
fig.show()

# OLD


In [ ]:
optimized_biomass_pandas = (
    viewer.best_simulation.pint.quantify()
    .pint.to("mg/meter^2")
    .pint.dequantify()
    .to_dataframe()
    .pivot_table(index="time", columns="functional_group", values="biomass")
)
original_biomass_pandas = (
    viewer.original_simulation.pint.quantify()
    .pint.to("mg/meter^2")
    .pint.dequantify()
    .to_dataframe()
    .pivot_table(index="time", columns="functional_group", values="biomass")
)
observations_day_pandas = (
    observations_selected_without_init.pint.quantify()
    .pint.to("mg/meter^2")
    .pint.dequantify()
    .day.dropna("time")
    .to_dataframe()
    .reset_index()
    .set_index("time")["day"]
)
observations_night_pandas = (
    observations_selected_without_init.pint.quantify()
    .pint.to("mg/meter^2")
    .pint.dequantify()
    .night.dropna("time")
    .to_dataframe()
    .reset_index()
    .set_index("time")["night"]
)
layer_pandas = epi_layer_depth.pint.dequantify().to_dataframe().reset_index().set_index("time")["pelagic_layer_depth"]

Then resample to month.


In [ ]:
monthly_obs_day = observations_day_pandas.resample("ME").mean()[TIME_START:TIME_END].dropna()
monthly_obs_day.index = monthly_obs_day.index.to_period("M").to_timestamp()

monthly_obs_night = observations_night_pandas.resample("ME").mean()[TIME_START:TIME_END].dropna()
monthly_obs_night.index = monthly_obs_night.index.to_period("M").to_timestamp()

monthly_pred_d1n1 = optimized_biomass_pandas.iloc[:, 0].resample("ME").mean()[TIME_START:TIME_END].dropna()
monthly_pred_d1n1.index = monthly_pred_d1n1.index.to_period("M").to_timestamp()

monthly_pred_d2n1 = optimized_biomass_pandas.iloc[:, 1].resample("ME").mean()[TIME_START:TIME_END].dropna()
monthly_pred_d2n1.index = monthly_pred_d2n1.index.to_period("M").to_timestamp()

monthly_pred_orignal = original_biomass_pandas.iloc[:, 0].resample("ME").mean()[TIME_START:TIME_END].dropna()
monthly_pred_orignal.index = monthly_pred_orignal.index.to_period("M").to_timestamp()

monthly_layer = layer_pandas.resample("ME").mean()[TIME_START:TIME_END].dropna()
monthly_layer.index = monthly_layer.index.to_period("M").to_timestamp()

In [ ]:
diagram = ModTaylorDiagram()

all_model = [monthly_pred_d1n1, monthly_pred_d1n1 + monthly_pred_d2n1, monthly_pred_orignal, monthly_pred_orignal]
all_obs = [monthly_obs_day, monthly_obs_night, monthly_obs_day, monthly_obs_night]

all_names = ["HOT Day", "HOT Night", "Original Day", "Original Night"]

for model, obs, name in zip(all_model, all_obs, all_names):
    diagram = generate_mod_taylor_diagram(diagram, obs=obs, model=model[obs.index], name=name)
diagram.plot()
plt.title(
    "Taylor Diagram for Seapodym model during day at HOT station with CAFE NPP : all parameters optimization and 2 groups"
)

# export the figure
if SAVE:
    plt.savefig(f"Taylor_{export_file_name}.png")

plt.show()

In [ ]:
if SAVE:
    diagram.get_stats().to_csv(f"Stats_{export_file_name}.csv", index=False)
diagram.get_stats()